**Подготовка к обучению моделей**

Импорт необходимых библиотек

In [8]:
import joblib
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

Загрузка данных и отбор признаков

In [9]:
# загрузка датасета
dataframe = pd.read_csv('data/dataset.csv')


x = dataframe.drop('custom_target', axis=1)
y = dataframe['custom_target']

# категориальные признаки
categorical_features = x.nunique()[3:].index.to_list()
# числовые признаки
numeric_features = x.nunique()[:2].index.to_list()

# для повторяемости
random_state = 77
#  разделяем  выборку на тестовую (20%), обучающую (60%) и валидационную (20%)
x_train_val, x_test, y_train_val, y_test = train_test_split(
    x, y,
    test_size=0.2,
    stratify=y,
    random_state=random_state
)

x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val,
    test_size=0.25,  # четверть обучающей выборки
    stratify=y_train_val,
    random_state=random_state
)

**Baseline**

In [10]:
dummy = DummyClassifier(strategy="prior")
dummy.fit(x_train, y_train)
roc_auc = roc_auc_score(y_test, dummy.predict_proba(x_test)[:, 1])
print(f"Значение метрики ROC-AUC для Baseline: {roc_auc:.4f}")

Значение метрики ROC-AUC для Baseline: 0.5000


**Логистическая регрессия**

В качестве первой модели мы взяли логистическую регрессию. Будем использовать прямое кодирование (One Hot Encoding) для категориальных признаков и стандартизацию (Standard Scaler) для числовых

In [11]:
model = Pipeline([
    ('data_preprocessing', ColumnTransformer(
        transformers=[
            ('numeric', StandardScaler(), numeric_features),
            ('categorical', OneHotEncoder(), categorical_features)
        ])
     ),
    ('classification', LogisticRegression(
        class_weight='balanced',
        random_state=random_state
    ))
])

Обучение модели

In [12]:
model.fit(x_train_val, y_train_val)

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('numeric', StandardScaler(),
                                                  ['go_to_car_card',
                                                   'pagination_click']),
                                                 ('categorical',
                                                  OneHotEncoder(),
                                                  ['photos_all', 'quiz_show',
                                                   'sap_search_form_cost_from',
                                                   'sap_search_form_cost_to',
                                                   'search_form_mark_select',
                                                   'search_form_model_select',
                                                   'search_form_region',
                                                   'search_form_search_btn',
                                                   'search_form_search_car_type_select',
                                                   'search_kpp',
                                                   'showed_number_ads',
                                                   'sub_car_page',
                                                   'sub_landing',
                                                   'sub_view_cars_click',
                                                   'view_card',
                                                   'view_more_click',
                                                   'view_new_card'])])),
                ('classification',
                 LogisticRegression(class_weight='balanced', random_state=77))])

Оценка модели линейной регрессии (ROC-AUC)

In [24]:
y_pred = model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_pred)
print(f"Значение метрики ROC-AUC для линейной регрессии: {roc_auc}")

Значение метрики ROC-AUC для линейной регрессии: 0.999865418025595


Сохранение модели

In [15]:
joblib.dump(model, 'model/logistic_regression.joblib')

['model/logistic_regression.joblib']

**Градиентный бустинг**

In [18]:
params = {
    'n_estimators': 150,
    'learning_rate': 0.2,
    'max_depth': 3,
    'min_samples_split': 20,
    'min_samples_leaf': 10,
    'max_features': 'sqrt',
    'subsample': 0.8,
    'random_state': random_state
}

model = Pipeline([
    ('data_preprocessing', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(), categorical_features)
        ])
     ),
    ('classification', GradientBoostingClassifier(**params))
])

Обучение модели

In [19]:
model.fit(x_train_val, y_train_val)

Pipeline(steps=[('data_preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['go_to_car_card',
                                                   'pagination_click']),
                                                 ('cat', OneHotEncoder(),
                                                  ['photos_all', 'quiz_show',
                                                   'sap_search_form_cost_from',
                                                   'sap_search_form_cost_to',
                                                   'search_form_mark_select',
                                                   'search_form_model_select',
                                                   'search_form_region',
                                                   'search_form_search_btn',
                                                   'search_form_search_car_type_select',
                                                   'search_kpp',
                                                   'showed_number_ads',
                                                   'sub_car_page',
                                                   'sub_landing',
                                                   'sub_view_cars_click',
                                                   'view_card',
                                                   'view_more_click',
                                                   'view_new_card'])])),
                ('classification',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            max_features='sqrt',
                                            min_samples_leaf=10,
                                            min_samples_split=20,
                                            n_estimators=150, random_state=77,
                                            subsample=0.8))])

Оценка модели линейной регрессии (ROC-AUC)

In [23]:
y_pred = model.predict(x_test)
roc_auc = roc_auc_score(y_test, y_pred)
print(f"Значение метрики ROC-AUC для градиентного бустинга: {roc_auc}")

Значение метрики ROC-AUC для градиентного бустинга: 0.999865418025595


Сохранение модели

In [22]:
joblib.dump(model, 'model/gradient_boosting.joblib')

['model/gradient_boosting.joblib']